In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using Theano backend.


In [60]:
#path = "/home/juneki/Documents/School/1_ComputerMusicSystems/project/encoded/DaftPunk_-_AroundTheWorld__Intricacy_20120430141138.gro.encoded"
path = "/home/juneki/Documents/School/1_ComputerMusicSystems/project/encoded/4BROSRVG.gro.encoded"


text = open(path).read().lower()

print "corpus length:", len(text)
print "TEXT BEFORE:", text[:80], "..."

text = text.split()
newtext = []
for word in text:
    word = word.split("|")
    word = tuple([float(word[0]), int(word[1]), int(word[2]), float(word[3]), int(word[4]), int(word[5])])
    newtext.append(word)
text = newtext

print "TEXT AFTER:", text[:3], "..."

words = set(text)
print "total words:", len(words)

corpus length: 206615
TEXT BEFORE: 1.0|37|37|1|84|0 0.5|37|37|1|84|0 0.5|37|37|1|64|0 0.25|37|37|1|84|0 0.25|37|37| ...
TEXT AFTER: [(1.0, 37, 37, 1.0, 84, 0), (0.5, 37, 37, 1.0, 84, 0), (0.5, 37, 37, 1.0, 64, 0)] ...
total words: 3492


In [61]:
maxlen = 10
step = 3
sentences,next_words = [],[]
#print text

#for i in range(0, len(text) - maxlen, step):
#    sentences.append(text[i: i + maxlen])
#    next_words.append(text[i + maxlen])
for i in range(0, len(text) - maxlen, step):
    
    #sentence = [[x for x in word.split("|")] for word in text[i: i + maxlen]]
    sentence = []
    for word in text[i: i+maxlen]:
        sentence.append(word)
    
    next_word = text[i + maxlen]
    next_words.append(next_word)
    sentences.append(sentence)
    
    
print sentences[:3], "..."
print "nb sequences:", len(sentences)
print

print "Vectorization"
word_indices = dict((w, i) for i,w in enumerate(words))
indices_words = dict((i, w) for i,w in enumerate(words))

#X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
#y = np.zeros((len(sentences), len(words)), dtype=np.bool)

X = np.zeros((len(sentences), maxlen, 6))
#y = np.zeros((len(sentences), 6))
y = np.zeros((len(sentences), len(words)), dtype=np.bool)

#print "X:", str(len(X))+"x"+str(len(X[0]))+"x"+str(len(X[0][0]))
#print "y:", str(len(y))+"x"+str(len(y[0]))

for i, sentence in enumerate(sentences):
    for t,word in enumerate(sentence):
        X[i, t] = list(word)
    #y[i] = list(next_words[i])
    
    #print next_words[i]
    y[i, word_indices[next_words[i]]] = 1
    
print
print "X:", str(len(X))+"x"+str(len(X[0]))+"x"+str(len(X[0][0]))
print "y:", str(len(y))+"x"+str(len(y[0]))

 
print X[0]
print y[0]



# for i, sentence in enumerate(sentences):
#     for t,word in enumerate(sentence):
#         X[i, t, word_indices[word]] = 1
#     y[i, word_indices[next_words[i]]] = 1


[[(1.0, 37, 37, 1.0, 84, 0), (0.5, 37, 37, 1.0, 84, 0), (0.5, 37, 37, 1.0, 64, 0), (0.25, 37, 37, 1.0, 84, 0), (0.25, 37, 37, 1.0, 64, 0), (0.25, 37, 37, 1.0, 84, 0), (0.25, 34, 34, 0.583333, 95, 0), (0.0, 56, 56, 1.49167, 70, 0), (0.0, 72, 72, 1.49167, 70, 0), (0.0, 62, 62, 1.49167, 70, 0)], [(0.25, 37, 37, 1.0, 84, 0), (0.25, 37, 37, 1.0, 64, 0), (0.25, 37, 37, 1.0, 84, 0), (0.25, 34, 34, 0.583333, 95, 0), (0.0, 56, 56, 1.49167, 70, 0), (0.0, 72, 72, 1.49167, 70, 0), (0.0, 62, 62, 1.49167, 70, 0), (0.0, 65, 65, 1.49167, 70, 0), (0.0, 72, 72, 0.666667, 94, 0), (0.0, 68, 68, 0.666667, 81, 0)], [(0.25, 34, 34, 0.583333, 95, 0), (0.0, 56, 56, 1.49167, 70, 0), (0.0, 72, 72, 1.49167, 70, 0), (0.0, 62, 62, 1.49167, 70, 0), (0.0, 65, 65, 1.49167, 70, 0), (0.0, 72, 72, 0.666667, 94, 0), (0.0, 68, 68, 0.666667, 81, 0), (0.0, 65, 65, 0.666667, 81, 0), (0.0, 62, 62, 0.666667, 81, 0), (0.0, 51, 51, 0.208333, 115, 0)]] ...
nb sequences: 2839

Vectorization

X: 2839x10x6
y: 2839x3492
[[  1.        

In [62]:
print "Build Model"

model = Sequential()
#model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(words))))
model.add(LSTM(512, return_sequences=False, input_shape=(maxlen, 6)))
model.add(Dropout(0.2))

#model.add(LSTM(512, return_sequences=False))
#model.add(Dropout(0.2))


model.add(Dense(len(words)))
#model.add(Dense(6))
model.add(Activation("softmax"))
#model.add(Activation("relu"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
#model.compile(loss="mse", optimizer="sgd")

print "Built Model"

Build Model
Built Model


In [38]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array                                                                                                        
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

def reencode(word):
    word = "|".join([str(x) for x in word])
    return word

In [ ]:
for iteration in range(0, 60):
    print
    print "-"*50
    print "Iteration", iteration
    
    
    print len(X), len(y)
    print len(X[0]), len(y[0])
    model.fit(X, y, batch_size=128, nb_epoch=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    #print "start_index:", start_index, text[start_index]
    
    
    #for diversity in [0.2, 0.5, 1.0, 1.2]:
    for diversity in [1.0, 2.0]:
        f = open("iter" + str(iteration) + "d:" + str(diversity) + ".output.encoded", "w")
        #f = sys.stdout
        
        #print
        #print "----- diversity:", diversity
        
        generated = ""
        sentence = text[start_index: start_index + maxlen]
        generated += " ".join([reencode(x) for x in sentence])
        #print "----- Generating with seed: ", sentence
        f.write(generated + " ")
        
        for i in range(400):
            #x = np.zeros((1, maxlen, len(words)))
            x = np.zeros((1, maxlen, 6))
            
            
            for t,word in enumerate(sentence):
                #print t,word
                
                #print 0, t, word_indices[word]
                #x[0, t, word_indices[word]] = 1
                x[0, t] = word
                
            #break
            
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_words[next_index]
            
            generated += reencode(next_word) + " "
            #generated += reencode(preds) + " "
            
            
            #next_word = preds
            sentence = sentence[1:] + [next_word]
            f.write(reencode(next_word) + " ")
            f.flush()
            
            
        #print
        
    


--------------------------------------------------
Iteration 0
2839 2839
10 3492
Epoch 1/1
2839/2839 [==============================] - 36s - loss: 6.5214    
start_index: 3613 (0.0, 56, 56, 2.78333, 72, 0)

----- Generating with seed:  [(0.0, 56, 56, 2.78333, 72, 0), (0.0, 71, 71, 2.78333, 72, 0), (0.0, 61, 61, 2.78333, 72, 0), (0.0, 64, 64, 2.78333, 72, 0), (0.0, 51, 51, 0.208333, 115, 0), (0.1667, 64, 64, 3.23333, 69, 0), (0.0, 61, 61, 3.23333, 62, 0), (0.0, 56, 56, 3.23333, 62, 0), (0.0, 52, 52, 3.33333, 56, 0), (0.0, 61, 61, 3.33333, 56, 0)]

----- Generating with seed:  [(0.0, 56, 56, 2.78333, 72, 0), (0.0, 71, 71, 2.78333, 72, 0), (0.0, 61, 61, 2.78333, 72, 0), (0.0, 64, 64, 2.78333, 72, 0), (0.0, 51, 51, 0.208333, 115, 0), (0.1667, 64, 64, 3.23333, 69, 0), (0.0, 61, 61, 3.23333, 62, 0), (0.0, 56, 56, 3.23333, 62, 0), (0.0, 52, 52, 3.33333, 56, 0), (0.0, 61, 61, 3.33333, 56, 0)]

--------------------------------------------------
Iteration 1
2839 2839
10 3492
Epoch 1/1
2839/283

In [42]:
a = (1,2,3)
print " ".join([str(x) for x in a])

print reencode(a)

1 2 3
1|2|3


In [44]:
print preds

[  6.83977050e-05   3.08672339e-01   3.00924957e-01   2.41073314e-03
   3.87915581e-01   7.97112807e-06]


In [55]:
a = sample(preds, diversity)
print indices_words[a]
print a

print preds


model.predict(x, verbose=0)
tuple(model.predict(x, verbose=0)[0])


(0.0188, 38, 38, 0.208333, 40, 0)
1
[ 1.05578721 -4.05843496  4.55931139  2.58586383 -3.71601248  0.32435885]


(1.055787205696106,
 -4.0584349632263184,
 4.5593113899230957,
 2.5858638286590576,
 -3.7160124778747559,
 0.32435885071754456)